In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "50g").config("spark.driver.maxResultSize",'0').appName('SparkImage').getOrCreate()

23/08/14 09:51:45 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface enp3s0)
23/08/14 09:51:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/14 09:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from PIL import Image, ImageDraw
from pyspark.sql.functions import lit
import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

In [3]:
df0 = spark.read.format("image").option("dropInvalid", True).load("SparkData/Dyskeratotic")
df0.select("image.origin", "image.width", "image.height", "image.nChannels", "image.mode", "image.data")
df_dys = df0.withColumn("label",lit(0))

df1 = spark.read.format("image").option("dropInvalid", True).load("SparkData/Koilocytotic")
df1.select("image.origin", "image.width", "image.height", "image.nChannels", "image.mode", "image.data")
df_koi = df1.withColumn("label",lit(1))

df2 = spark.read.format("image").option("dropInvalid", True).load("SparkData/Metaplastic")
df2.select("image.origin", "image.width", "image.height", "image.nChannels", "image.mode", "image.data")
df_meta = df2.withColumn("label",lit(2))

df3 = spark.read.format("image").option("dropInvalid", True).load("SparkData/Parabasal")
df3.select("image.origin", "image.width", "image.height", "image.nChannels", "image.mode", "image.data")
df_para = df3.withColumn("label",lit(3))

df4 = spark.read.format("image").option("dropInvalid", True).load("SparkData/Superficial-Intermediate")
df4.select("image.origin", "image.width", "image.height", "image.nChannels", "image.mode", "image.data")
df_super = df4.withColumn("label",lit(4))



In [4]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [df_dys,df_koi,df_meta,df_para,df_super]
df = reduce(DataFrame.unionAll, dfs)


In [5]:
df.show(5)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|{file:///home/thu...|    0|
|{file:///home/thu...|    0|
|{file:///home/thu...|    0|
|{file:///home/thu...|    0|
|{file:///home/thu...|    0|
+--------------------+-----+
only showing top 5 rows



In [6]:
ImageSchema.imageFields

img2vec = F.udf(lambda x: DenseVector(ImageSchema.toNDArray(x).flatten()), VectorUDT())

df = df.withColumn('features', img2vec("image"))
df = df.select("features", "label")
df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[73.0,59.0,61.0,8...|    0|
|[71.0,63.0,63.0,7...|    0|
|[58.0,29.0,68.0,6...|    0|
|[49.0,59.0,69.0,5...|    0|
|[79.0,53.0,67.0,8...|    0|
+--------------------+-----+
only showing top 5 rows



# Standard scaling

In [7]:
from pyspark.ml.feature import PCA
pca = PCA(k=1000, inputCol="features")

In [8]:
model=pca.fit(df)

23/08/06 06:42:04 WARN RowMatrix: 17787 columns will require at least 2531 megabytes of memory!
23/08/06 06:42:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Py4JJavaError: An error occurred while calling o131.fit.
: java.lang.RuntimeException: The param k and numFeatures is too large for SVD computation. Try reducing the parameter k for PCA, or reduce the input feature vector dimension to make this tractable.
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:91)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:47)
	at breeze.generic.UFunc.apply$(UFunc.scala:46)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:501)
	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:65)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:93)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java